# Main

In [ ]:
import os.path
import pickle
import re
import importlib
# importlib.reload(visualization) # this reloads changes to external scripts

filename = 'extracted_data.pickle'
if os.path.isfile(filename):
    with open(filename, 'rb') as handle:
        # this contains all extracted data
        curr_data_dict = pickle.load(handle)
else:
    print("File does not exist")

    # this contains all extracted data
    curr_data_dict = {}

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

# Initiate the browser
#browser = webdriver.Chrome(ChromeDriverManager().install())

chrome_options = Options()
#chrome_options.add_argument("--incognito")
#chrome_options.add_argument("--window-size=1920x1080")

driver = webdriver.Chrome(chrome_options=chrome_options, executable_path="./chromedriver.exe")

In [ ]:
from auction_lib.functions import get_auction_id, get_current_price, save_data_into_dict, \
                                  get_time_series, get_offer_metrics, get_auction_detail_id, \
                                  get_auction_detail_end_time

def open_main_auction_site(url_adress):
    # Open the Website
    driver.get(url_adress)

    # Click to show 50 elements
    #driver.find_element_by_id('resultListLimit').click()
    select = Select(driver.find_element_by_id('resultListLimit'))
    # select by visible text
    select.select_by_visible_text('50')
    # select by value
    select.select_by_value('50')

    # Click to reload with 50 elements
    driver.find_element_by_id('buttons').click()

In [ ]:
from datetime import datetime
import time

# loop until interrupt
try:
    while True:
        #print("Loop")
        now = datetime.now()
        extraction_time = now.strftime("%Y-%m-%d_%H:%M:%S")
        print("Current Time =", extraction_time)

        # Part 1 ===========================================
        # Open the Website
        url_adress = "https://www.justiz-auktion.de/Elektronik-%3EPC--Tablets~1858"
        open_main_auction_site(url_adress)
        time.sleep(0.1)

        # Part 2 ===========================================
        ##Find elements with Selenium, by css
        # use the chrome plugin Selector Gadget: https://chrome.google.com/webstore/detail/selectorgadget/mhjhnkcfbdhnjickkkdbjoemdmbfginb
        element_css_selector = ".spalte2"
        els = driver.find_elements_by_css_selector(element_css_selector)

        for i in range(0,len(els)):
            text = els[i].text

            # extracted information
            curr_id = get_auction_id(text)
            curr_price = get_current_price(text)

            #print(extraction_time, ":\t", curr_id, curr_price)
            curr_data_dict = save_data_into_dict(curr_data_dict, curr_id, extraction_time, curr_price)

        # Part 3 ===========================================
        # Get detailed information from each offer

        # iterate through all auctions on the main page
        min_time_left = 99999
        speedup_flag = False

        links = driver.find_elements_by_xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "mainpageLink", " " ))]')
        for link in links:
            if link.text == "Details":
                link_href = link.get_attribute('href')
                #print(link.text, link.get_attribute('href'))

                # open new tab
                driver.execute_script("window.open('');")
                driver.switch_to.window(driver.window_handles[1])

                driver.get(link_href)

                # extract detailed metrics from auction offer page
                element_css_selector = "#geb_uebersicht"
                el = driver.find_element_by_css_selector(element_css_selector)
                text = el.text

                highest_bidder, n_o_bids, n_o_clicks, n_o_watchers = get_offer_metrics(text)
                details_dict = {}
                details_dict["highest_bidder"] = highest_bidder
                details_dict["n_o_bids"] = n_o_bids
                details_dict["n_o_clicks"] = n_o_clicks
                details_dict["n_o_watchers"] = n_o_watchers

                curr_detailed_id = get_auction_detail_id(driver)
                curr_data_dict[curr_detailed_id][extraction_time]["details"] = details_dict

                # check time until auction end
                end_time = get_auction_detail_end_time(driver)
                date_time_obj = datetime.strptime(end_time, '%d.%m.%Y %H:%M:%S')
            
                # set speedup flag when we reach the end of the auction
                time_left = (date_time_obj - now).seconds
                if min_time_left>time_left:
                    min_time_left = time_left
                    if time_left > 3600:
                        speedup_flag = False
                    elif (time_left <= 3600) and (time_left > 600):
                        speedup_flag = 1
                    elif (time_left <= 600) and (time_left > 120):
                        speedup_flag = 2
                    elif (time_left <= 120):
                        speedup_flag = 3
                    else:
                        speedup_flag = 3
                else:
                    pass

                driver.close()
                driver.switch_to.window(driver.window_handles[0])
            else:
                driver.switch_to.window(driver.window_handles[0])

        # Part 4 ===========================================
        # get current value of XYZ
        XYZ_label = "XYZ"
        if XYZ_label not in curr_data_dict: curr_data_dict[XYZ_label] = {}
        curr_data_dict["XYZ"][extraction_time] = 0


        # Part 5 ===========================================
        # when auction end comes nearer, start to raise the refresh rate
        speedup_flag = 2

        if speedup_flag == False: time.sleep(300) # TODO change interrupt style, waiting 10min sucks
        elif speedup_flag == 1: time.sleep(60)
        elif speedup_flag == 2: time.sleep(10)
        elif speedup_flag == 3: time.sleep(1)
        else: time.sleep(60)

except KeyboardInterrupt:
    time.sleep(1)
    if len(driver.window_handles)>0:
        driver.switch_to.window(driver.window_handles[1])
        driver.close()
        driver.switch_to.window(driver.window_handles[0])

    pass

In [ ]:
curr_data_dict